In [ ]:
import sys
sys.path.append('figures')
sys.path.append('methods')


from pandas import DataFrame as DF
import glob
import json
from figures.route import *

## Load Data

In [ ]:
data_paths_targets = ['line_graph', 'complite_graph', 'babeld']

In [ ]:
class MetricsObj(object):

    def __init__(self, metrics_folder):
        metrics_path = "../data/routing/{}/*.json"
        for json_path in glob.glob(metrics_path.format(metrics_folder)):
            name_obj = json_path.split("/")[-1].split('.')[0]
            name_obj = 'dBm{}'.format(name_obj.split('d')[0])
            with open(json_path, 'r') as jsonfile_obj:
                metrics_dict = json.load(jsonfile_obj)
                setattr(self, name_obj, metrics_dict)

In [ ]:
line_graph_matrics_obj = MetricsObj('line_graph')
complite_graph_matrics_obj = MetricsObj('complite_graph')
babeld_matrics_obj = MetricsObj('babeld')

## show udp_throughput/time


In [ ]:
# complte graph
print("complte graph")

print("10dBm")
save_obj = ['complite_graph', "10dBm"]
udp_figure(complite_graph_matrics_obj.dBm10,save_obj=None)

print("20dBm")
save_obj = ['complite_graph', "20dBm"]
udp_figure(complite_graph_matrics_obj.dBm20, save_obj=None)

print("31dBm")
save_obj = ['complite_graph', "31dBm"]
udp_figure(complite_graph_matrics_obj.dBm31, save_obj=None)


# line graph
print("line graph")

print("10dBm")
save_obj = ['line_graph', "10dBm"]
udp_figure(line_graph_matrics_obj.dBm10,save_obj=None)

print("20dBm")
save_obj = ['line_graph', "20dBm"]
udp_figure(line_graph_matrics_obj.dBm20, save_obj=None)

print("31dBm")
save_obj = ['line_graph', "31dBm"]
udp_figure(line_graph_matrics_obj.dBm31, save_obj=None)


# babel
print("babeld")

print("10dBm")
save_obj = ['babeld', "10dBm"]
udp_figure(babeld_matrics_obj.dBm10, save_obj=None)

print("20dBm")
save_obj = ['babeld', "20dBm"]
udp_figure(babeld_matrics_obj.dBm20, save_obj=None)

print("31dBm")
save_obj = ['babeld', "31dBm"]
udp_figure(babeld_matrics_obj.dBm31, save_obj=None)

## show mean udp_throughput for every topology per dBm (thing for this)

In [ ]:
def avgsdBm_dataframe(matrics_obj):
    avgsdBm10 = []
    for values in matrics_obj.dBm10.values():
        avgsdBm10.append(np.average(values))

    avgsdBm20 = []
    for values in matrics_obj.dBm20.values():
        avgsdBm20.append(np.average(values))

    avgsdBm31 = []
    for values in matrics_obj.dBm31.values():
        avgsdBm31.append(np.average(values))

    avgsdBm = np.array([avgsdBm10,avgsdBm20,avgsdBm31])
    avgsdBm = DF(avgsdBm,
        index=["10 dBm", "20 dBm", "31 dBm"],
        columns=list(matrics_obj.dBm10.keys())
        )
    return avgsdBm


# complite graph
print("complite graph")
avgsdBm_df = avgsdBm_dataframe(complite_graph_matrics_obj)
print(avgsdBm_df)
save_obj = ['complite_graph', "mean-per-dBm"]
mean_udp_per_dBm_bar_figure(avgsdBm_df, save_obj=None)

# line graph
print("line graph")
avgsdBm_df = avgsdBm_dataframe(line_graph_matrics_obj)
print(avgsdBm_df)
save_obj = ['line_graph', "mean-per-dBm"]
mean_udp_per_dBm_bar_figure(avgsdBm_df, save_obj=None)

# babeld
print("babeld")
avgsdBm_df = avgsdBm_dataframe(babeld_matrics_obj)
print(avgsdBm_df)
save_obj = ['babeld', "mean-per-dBm"]
mean_udp_per_dBm_bar_figure(avgsdBm_df, save_obj=None)

## show mean udp_throughput per rpi per dBm for line_graph with mathmodel in all plots

In [ ]:
    

def means_per_txpower(matrics_obj):

    def mean_txpower(matrics_dict):
        avg = []
        for _, rpi_metrics in matrics_dict.items():
            avg.append(np.average(rpi_metrics))
    
        return avg

    # find way to get objects (dBm10,20,31...) for "for loop"
    avg10 = mean_txpower(matrics_obj.dBm10)
    avg20 = mean_txpower(matrics_obj.dBm20)
    avg31 = mean_txpower(matrics_obj.dBm31)

    avgs = np.concatenate([[avg10], [avg20], [avg31]],axis=0)
    return avgs




save_obj = ['comparison','3d-mean-per-rpi']
line = means_per_txpower(line_graph_matrics_obj)
complite = means_per_txpower(complite_graph_matrics_obj)
babeld = means_per_txpower(babeld_matrics_obj)
mean_udp_bar3d_figure(complite, line, babeld,save_obj=None)

In [ ]:
def means_model_vals(objdBm):
    means_vals = []
    model_vals = []
    model_val0 = None
    model_ratio = None

    for i, (key, value) in enumerate(objdBm.items()):
        mean_val = np.mean(value)
        means_vals.append(mean_val)
        h = i+1
        if key == 'RPI 0 (c) -- RPI 1 (s)':
            model_val0 = mean_val
            model_val = model_val0
            model_vals.append(model_val)
        elif key == 'RPI 0 (c) -- RPI 2 (s)':
            model_val1 = mean_val
            model_ratio = model_val0/model_val1
            model_val = model_val0*model_ratio**(1-h)
            model_vals.append(model_val)
        else:
            model_val = model_val0*model_ratio**(1-h)
            model_vals.append(model_val)

    
    return means_vals, model_vals, model_ratio


save_obj_base = ['line_graph']

# dBm10
print('dBm10')
save_obj = save_obj_base+['10dBm-model']
means_vals, model_vals, a = means_model_vals(line_graph_matrics_obj.dBm10)
model_feagure(means_vals, model_vals, legend_fontsize=28, markersize=15, axes_label_fontsize=28, ticks_fontsize=24, save_obj=None)
print("a:",a)

# dBm20
print('dBm20')
save_obj = save_obj_base+['20dBm-model']
means_vals, model_vals, a = means_model_vals(line_graph_matrics_obj.dBm20)
model_feagure(means_vals, model_vals, legend_fontsize=28, markersize=15, axes_label_fontsize=28, ticks_fontsize=24, save_obj=None)
print("a:",a)

# dBm31
print('dBm31')
save_obj = save_obj_base+['31dBm-model']
means_vals, model_vals, a = means_model_vals(line_graph_matrics_obj.dBm31)
model_feagure(means_vals, model_vals, legend_fontsize=28, markersize=15, axes_label_fontsize=28, ticks_fontsize=24, save_obj=None)
print("a:",a)
